In [1]:
import keras
import numpy as np
import tensorflow as tf
from tqdm.auto import tqdm, trange


# Prepare data

In [2]:
import unicodedata
import regex as re

def normalize_text(text):
    clean_text=text.lower()
    clean_text=re.sub(r"[^a-zA-Z0-9\säåö]"," ",clean_text)
    #clean_text=re.sub(r"[^\S\n]+"," ",clean_text)
    clean_text=re.sub(r"\s+"," ",clean_text)
    clean_text=re.sub(r"^\s","",clean_text)
    clean_text=re.sub(r"\s$","",clean_text)
    return clean_text

normalize_text("äåöjkjh jhjk j1hkj"),normalize_text("jFjh.jhjk,.,.ef jhkj "),normalize_text(" jkjh jhjk jhkj")

('äåöjkjh jhjk j1hkj', 'jfjh jhjk ef jhkj', 'jkjh jhjk jhkj')

In [ ]:
import csv
finnish_company_names=[]
with open('fullprhdata.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=';')
    for row in csv_reader:
        finnish_company_names.append(row[0])
json.dump(finnish_company_names,open("finnish_registry_companies.json","w+"))

In [3]:
import json 
import random
finnish_companies = json.load(open("finnish_registry_companies.json","r+"))
all_companies = json.load(open("all_kaggle_companies.json","r+"))

names_text=""

# companies 
#names_arr=[normalize_text(company) for company in tqdm(all_companies)]
#random.shuffle(names_arr)
#names_text="\n".join(names_arr)

# finnish 
finnish_names_arr=[normalize_text(company) for company in tqdm(finnish_companies)]
random.shuffle(finnish_names_arr)
finnish_names_text="\n".join(finnish_names_arr)


characters=sorted(list(set(names_text+finnish_names_text)))
characters=["."]+characters # . is none
char_indices = dict((c, i) for i, c in enumerate(characters))
indices_char = dict((i, c) for i, c in enumerate(characters))

Tx=25


In [4]:
print("Characters:","".join(characters))
print("Chars: ",len(characters))
print("Tx: ",Tx)

Characters: .
 0123456789abcdefghijklmnopqrstuvwxyzäåö
Chars:  42
Tx:  25


In [5]:
def str_to_vec(word, start_with_null=False):
    """
    Converts word to vec
    
    word -- string
    
    returns array of shape (Tx, len(chars))
    """
    if start_with_null:
        word="."+word
    x = np.zeros((len(word), len(characters)), dtype=np.bool)
    for t, char in enumerate(word):
        x[t, char_indices[char]] = 1
    return x
    
def vec_to_str(vec):
    """
    Converts vec to word
    
    vec -- array of shape (Tx, len(chars))
    
    """
    word=""
    for i in range(vec.shape[0]):
        word+=indices_char[np.argmax(vec[i])]
    return word

a=str_to_vec("hello world",start_with_null=True)
#print(a)
print(vec_to_str(a))

.hello world


In [7]:
id_=3430
vec_to_str(Xf[id_]),vec_to_str(np.array(Yf[id_]))

NameError: name 'Xf' is not defined

# Model

In [ ]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    out = np.random.choice(range(len(characters)), p = probas.ravel())
    out=characters[out]
    return out

def generate_output(model, text_start, length=5):
    
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print(f"Diversity: {diversity}")
        for l in range(length):
            generated = '\n'
            sentence = ('{0:\n>' + str(Tx) + '}').format(text_start).lower()
            generated += text_start 
            gens=0

            generated = ""
            inp=text_start
            sentence = inp

            x_pred = np.zeros((1, Tx, len(characters)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.0
            preds = model.predict(x_pred, verbose=0)[0]
            for pred in preds:
                next_index = sample(pred)
                next_char = next_index
                sentence = sentence + next_char
                generated += next_char
            print(f"{inp}{generated}")
        
generate_output(model,"accent")

In [ ]:
from keras import layers, models
from keras.layers import Dense, Input, LSTM,GRU

drp=0
model=models.Sequential()
model.add(LSTM(128, input_shape=(Tx, len(characters)),return_sequences=True))
model.add(Dense(len(characters),activation="softmax"))
model.summary()

In [ ]:
from keras.callbacks import LambdaCallback

def on_epoch_end(epoch, logs):
    generate_output(model,"acc")

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)


In [ ]:
model.compile(loss='categorical_crossentropy', optimizer="adam")

history=model.fit(X, Y, batch_size=4096, validation_split=0.1,  epochs=100, shuffle=True, callbacks=[print_callback])

In [ ]:
model.save("alltext.hdf5")

# Per name model

In [ ]:
import json 
import random
finnish_companies = json.load(open("finnish_registry_companies.json","r+"))

Tx=15

# finnish 
finnish_names_arr=[normalize_text(company) for company in finnish_companies if len(company)<Tx]
random.shuffle(finnish_names_arr)
finnish_names_text="".join(finnish_names_arr)

characters=sorted(list(set(finnish_names_text)))
characters=[".","E"]+characters # . is none, E is end of line
char_indices = dict((c, i) for i, c in enumerate(characters))
indices_char = dict((i, c) for i, c in enumerate(characters))

print(len(finnish_names_arr))
print("\n".join(random.choices(finnish_names_arr,k=10)))

In [ ]:
def vectorization(words, n_x, Tx=None):
    """
    Convert X and Y (lists) into arrays to be given to a recurrent neural network.
    
    Arguments:
    X -- 
    Y -- 
    Tx -- integer, sequence length
    
    Returns:
    x -- array of shape (m, Tx, len(chars))
    y -- array of shape (m, Tx, len(chars))
    """
    if Tx is None:
        Tx=len(max(words,key=len))
        print(Tx)
    
    m = len(words)
    x = np.zeros((m, Tx, n_x), dtype=np.bool)
    y = np.zeros((m, Tx, n_x), dtype=np.bool)
    
    for w, word in enumerate(tqdm(words)):
        word=word[:Tx]
        x[w, 0:len(word)+1, :] = str_to_vec(word,start_with_null=True)
        x[w, len(word)+1:, char_indices["E"]] = 1
        
        y[w, 0:len(word),:] = str_to_vec(word)
        y[w, len(word):, char_indices["E"]] = 1
        
    return x, y 

Xf,Yf=vectorization(finnish_names_arr, n_x=len(characters), Tx=Tx)
Xf.shape, Yf.shape 

In [ ]:

print(vec_to_str(Xf[11324])),
print(vec_to_str(Yf[11324]))

In [ ]:
def make_name(model, beginning=""):
    name = beginning
    x = np.zeros((1, Tx, len(characters)))
    x[0,0:len(beginning),:]=str_to_vec(beginning)
    
    for i in range(len(beginning)-1,Tx-1):
        prediction=model.predict(x)[0]
        probs = list(prediction[i])
        probs = probs / np.sum(probs)
        index = np.random.choice(range(len(characters)), p=probs)
        #index = np.argmax(probs)
        print(index)
        character = indices_char[index]
        print(f"{vec_to_str(x[0])} -> {vec_to_str(prediction)} -> {character}")
        if character=="E":
            break
        name+=character
        x[0, i+1, index] = 1

        i += 1
    
    print(name)

make_name(model)

In [ ]:
sequence.pad_sequences([[0,0,0],[0,0,0],[0,0,0]], maxlen=10, value=[0,0,0])

In [ ]:
from keras import layers, models
from keras.layers import Dense, Input, LSTM,GRU

drp=0
model=models.Sequential()
model.add(LSTM(64, input_shape=(Tx, len(characters)), return_sequences=True))
model.add(Dense(len(characters),activation="softmax"))
model.summary()

In [ ]:
from keras.callbacks import LambdaCallback

def on_epoch_end(epoch, logs):
    for i in range(3):
        make_name(model,"em")
        
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)


In [ ]:
opt=keras.optimizers.Adam(learning_rate=0.0001)

model.compile(loss='categorical_crossentropy', optimizer="adam")

history=model.fit(Xf, Yf, batch_size=8192, validation_split=0.1,  epochs=1000, shuffle=True, callbacks=[print_callback],verbose=1)

In [ ]:
import matplotlib
from matplotlib import pyplot as plt

results=history

plt.figure(figsize=(8, 16))
plt.title("Learning curve")
plt.plot(results.history["loss"], label="loss")
plt.plot(results.history["val_loss"], label="val_loss")
plt.plot( np.argmin(results.history["val_loss"]), np.min(results.history["val_loss"]), marker="x", color="r", label="best model")
plt.xlabel("Epochs")
plt.ylabel("log_loss")
plt.legend();


In [ ]:
from keras.preprocessing import sequence

empty="."
eos="E"
eos_index=char_indices[eos]
xn=len(characters)


def predict_tree(x, position, k):
    pred=model.predict(x)[0][position]
    indices = pred.argsort()[-k:]
    results=np.zeros((0, Tx, xn))
    for index in indices:
        res=np.array(x, copy=True)
        res[0,position,index]=pred[index]
        if index==eos_index:
            results=np.append(results,res,axis=0)
            break
        if position==Tx-1:
            results=np.append(results,res,axis=0)
            break
        results=np.append(results,predict_tree(res,position+1,k), axis=0)
    return results


def beamsearch(k, cnt):
    x = np.zeros((1, Tx, xn))
    res=predict_tree(x, 0, k)
    probs=np.sum(res,axis=(1,2))
    samples=[vec_to_str(a) for a in np.take(res,np.argsort(probs),axis=0)[-k:]]
    return list(zip(probs,samples))
    
    
res=beamsearch(k=2,cnt=10)
for prob, sample in res:
    print(f"{probs:0.2f}: {sample}")

In [ ]:
# beam search
# train language model for the beam search
# end of line character should be zeros
# dropout
# try GRU
# save best model



In [ ]:
from keras import layers, models
from keras.layers import Dense, Input, LSTM,GRU

drp=0
model=models.Sequential()
model.add(LSTM(64, input_shape=(Tx, len(characters)),return_sequences=True,activation="softmax"))
model.add(Dense(len(characters),activation="softmax"))
model.summary()

In [ ]:
np.sum(model.predict(Xf[2:3,:,:]), axis=2)

In [ ]:
model.predict(Xf[:1,:,:])